In [2]:
import os
import sys
sys.path.append('..')
from backtest.operators import * 
from data_center.tej_handler import TEJHandler
from monitor.tw_monitor import TWMarketMonitor

from pymongo import MongoClient
import tejapi

import pandas as pd
import numpy as np
from datetime import date
from dateutil.relativedelta import relativedelta
# TEJHandler()

### Get Weighting

In [3]:
collection = "monthly_revenue"
monthly_rev_pmart = TWMarketMonitor().directly_get_pmart(collection)

In [4]:
monthly_rev_pmart['revenue_yoy'] = monthly_rev_pmart['單月營收(千元)'] / monthly_rev_pmart['去年單月營收(千元)'] -1
example_factor = ts_rank(monthly_rev_pmart['revenue_yoy'],252)

In [5]:
def get_weighting(factor):
    demean = factor.sub(factor.mean(axis=1), axis=0)
    weighting = demean.div(demean.abs().sum(axis=1), axis=0)
    return weighting

### 假設出訊號後隔天收盤進場

In [17]:
example_signal = get_weighting(example_factor)
example_weighting = example_signal.shift(2) #隔天收盤進場，後天才有部位

### Get returns

In [8]:
collection = 'securities_returns'
returns_pmart = TWMarketMonitor().directly_get_pmart(collection)

In [15]:
collection = 'securities_trading_data'
ohlcv_pmart = TWMarketMonitor().directly_get_pmart(collection)

In [19]:
example_returns = returns_pmart['日報酬率(%)']

### 將weighting 與 returns結合獲得績效

In [25]:
alpha_ret = example_weighting * example_returns
recent_weekly_ret =  alpha_ret[-5:].dropna(axis =1, how = 'all')

In [27]:
recent_weekly_ret.sum(axis =1)

datetime
2024-03-28    0.022082
2024-03-29    0.037265
2024-04-01    0.115265
2024-04-02    0.121972
2024-04-03    0.005566
dtype: float64